In [1]:
#First test for git aliases (e.g., git-update-plus, git-next-checkin, etc)

# Note: The aliases should be defined before Jupyter is invoked.
# For BatsPP, they can loaded via '--source all-tomohara-aliases-etc.bash' option.

In [2]:
## Lorenzo-note:
## Be very careful to ensure that the output is reproducible.
## This can be done by encoding numbers as <num>, hex sequences as <hex>, etc.
## In addition, make sure standard error is redirected so that it doesn't confuse BatsPP.

In [3]:
# Global Setup
# Creating temp local repos to test on
#
## Lorenzo-TODO3: Use TMPA for first case for consistency and becuase you are not changing system temp directory.
TMP=/tmp/temp-git-test
TMPB=/tmp/temp-git-test-B

In [4]:
# Global setup
# Define aliases for encoding output (e.g., to ensure output the same across runs).
## OLD: alias encode-control-chars='perl -pe "s/\t/<tab>/g;  s/\n/<nl>/g;  s/[\x00-\x31]/<cc>/g;"'
alias encode-control-chars='perl -pe "s/\t/<tab>/g;  s/[\x00-\x08\x11\x31]/<cc>/g;"'
alias encode-nums='perl -pe "s/[0-9]+/N/g;"'
## TODO: define a better hex encoder to avoid false positives (e.g., abc).
alias encode-hex='perl -pe "s/[0-9A-F]{4,}/<hex>/ig;"'
function encode-text () { encode-control-chars | encode-nums | encode-hex; }

In [5]:
# Global Setup
## Lorenzo-Note: BatsPP has a problem with commands mixed with comments.
# Warning: Due to BatpPP quirk, comments should not be placed within command sequence.
# So make sure there is only one command sequence per cell
## OLD:
## git clone https://github.com/tomasohara/git-bash-test.git "$TMP"
## git clone https://github.com/tomasohara/git-bash-test.git "$TMPB"
## Lorenzo-note: output ignored due to difficulty obfuscating and since this is just a setup
git clone https://github.com/tomasohara/git-bash-test.git "$TMP" > /dev/null 2>&1
git clone https://github.com/tomasohara/git-bash-test.git "$TMPB" > /dev/null 2>&1

In [6]:
# Global Setup
# Move to second local to push some changes
command cd "$TMPB" 
git pull origin main 
echo "hello second local" > hello.txt
git add hello.txt 2>&1 | encode-text
git commit -m "first commit on second local" 2>&1 | encode-text
git push 2>&1 | encode-text

From https://github.com/tomasohara/git-bash-test
 * branch            main       -> FETCH_HEAD
Already up to date.
[main NcaNdc] first commit on second local
 <cc> file changed, <cc> insertion(+)
 create mode <cc>N hello.txt
To https://github.com/tomasohara/git-bash-test.git
   <hex>N..NcaNdc  main -> main


In [7]:
# Moving to first local
command cd "$TMP"

In [8]:
# git-branch-alias test
# switch to new branch

git checkout -b newBranch;
actual_branch=$(git branch --show-current)


Switched to a new branch 'newBranch'


In [9]:
# Assert
[ $(echo $actual_branch) == "$(git-branch-alias)" ]; echo $?;

0


In [10]:
#git-checkout-branch test
git-checkout-branch --help


usage: git-checkout-branch [--help | branch]
note: available branches:
    main


In [11]:
git-checkout-branch main
actual_branch=$(git branch --show-current)

issuing: git checkout main --
Switched to branch 'main'
Your branch is up to date with 'origin/main'.


In [12]:
# Assert
[ $(echo $actual_branch) == "main" ]; echo $?;

0


In [13]:
# git-update-plus test
# *** Make sure you obfuscare the output; also make sure no control characters passed ***
## BAD: git-update-plus
git-update-plus 2>&1 | encode-text

No changed files so no pre-stash-pop timestamp check
issuing: git stash
issuing: git pull --all
From https://github.com/tomasohara/git-bash-test
   <hex>N..NcaNdc  main       -> origin/main
issuing: git stash pop
No stash entries found.
No changed files so no post-stash-pop timestamp check
No local changes to save
Fetching origin
Updating <hex>N..NcaNdc
Fast-forward
 hello.txt | <cc> +
 <cc> file changed, <cc> insertion(+)
 create mode <cc>N hello.txt



In [14]:
# Assert
[ "$(cat hello.txt)" == "hello second local" ]; echo $?;

0


In [15]:
# Setup
# adding changes to not be able to pull
command cd "$TMPB"
echo "second change" > hello2.txt
git add . 2>&1 | encode-text
git commit -m "second commit" 2>&1 | encode-text
git push 2>&1 | encode-text

command cd "$TMP"
echo "second line" >> hello.txt

[main NcNaN] second commit
 <cc> file changed, <cc> insertion(+)
 create mode <cc>N helloN.txt
To https://github.com/tomasohara/git-bash-test.git
   NcaNdc..NcNaN  main -> main


In [16]:
# Should fail
git-update-plus

Error: Use 'git-update-force' to update with changed files (n.b., potential for conflicts)


In [17]:
# git-update-force test 
# should pull
## OLD: git-update-force
## Lorenzo-note: hides the lines with hex
git-update-force 2>&1 | encode-text

issuing: zip over changed files (for later restore)
issuing: git stash
issuing: git pull --all
From https://github.com/tomasohara/git-bash-test
   NcaNdc..NcNaN  main       -> origin/main
issuing: git stash pop
issuing: unzip over _stash.zip (to restore timestamps)
no changes <hex> to commit (use "git add" and/or "git commit -a")
Dropped refs/stash@{N} (Nb<cc>N<hex>NbNcNefdNd<cc>cdNfNceN)
unzip -v -o _stash.zip
Archive:  _stash.zip
 Length   Method    Size  Cmpr    Date    Time   CRC-N   Name
--------  ------  ------- ---- ---------- ----- --------  ----
      N<cc>  Defl:N       N  <cc>N% N-<cc>N-N N<cc>:N NbecN  hello.txt
--------          -------  ---                            -------
      N<cc>               N  <cc>N%                            <cc> file



In [18]:
#assert
[ "$(cat hello2.txt)" == "second change" ]; echo $?;

0


In [19]:
#git-commit-and-push test
echo "pushed" > push.txt
## OLD: git status
git status 2>&1 | encode-text

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
<tab>modified:   hello.txt

Untracked files:
  (use "git add <file>..." to include in what will be committed)
<tab>_git-diff-list.<cc><cc>N<cc><cc>.bash
<tab>_git-diff.<cc><cc>N<cc><cc>.list
<tab>_stash.zip
<tab>push.txt

no changes <hex> to commit (use "git add" and/or "git commit -a")


In [20]:
export GIT_MESSAGE="hello"
previous=$GIT_NO_CONFIRM
export GIT_NO_CONFIRM="1"
## OLD: git-commit-and-push push.txt
git-commit-and-push push.txt 2>&1 | encode-text

issuing: git add push.txt

GIT_NO_CONFIRM set to <cc>, so skipping confirmation
issuing: git commit -m 'hello'
    [main NbN] hello
     <cc> file changed, <cc> insertion(+)
     create mode <cc>N push.txt
GIT_NO_CONFIRM set to <cc>, so skipping confirmation
issuing: git push --verbose
Pushing to https://github.com/tomasohara/git-bash-test.git
POST git-receive-pack (N bytes)
To https://github.com/tomasohara/git-bash-test.git
   NcNaN..NbN  main -> main
updating local tracking ref 'refs/remotes/origin/main'
[main NbN] hello
 <cc> file changed, <cc> insertion(+)
 create mode <cc>N push.txt



In [21]:
command cd "$TMPB"
git pull 2>&1 | encode-text

From https://github.com/tomasohara/git-bash-test
   NcNaN..NbN  main       -> origin/main
Updating NcNaN..NbN
Fast-forward
 push.txt | <cc> +
 <cc> file changed, <cc> insertion(+)
 create mode <cc>N push.txt


In [22]:
#assert
[ "$(cat push.txt)" == "pushed" ]; echo $?;

0


In [23]:
# undo changes and remove newly created files

## Lorenzo-TODO2: add more output encoding
## also, converting unique ID's to codes like temp-git-test/.git/objects/pack/pack-HEX.idx
## EX: cat $log | perl -pe 's/\b[0-9]+\b/NUM/g; s/\b[a-f0-9]+\b/HEX/g;'
rm -f *.txt > /dev/null
git rm -f *.txt > /dev/null
git add . > /dev/null

In [24]:
## Lorenzo-TODO2: use one cell per output (to make this less brittle)
git commit -m "cleanup test files" 2>&1 | encode-text
git push 2>&1 | encode-text

[main NbNaN] cleanup test files
 N files changed, N deletions(-)
 delete mode <cc>N hello.txt
 delete mode <cc>N helloN.txt
 delete mode <cc>N push.txt
To https://github.com/tomasohara/git-bash-test.git
   NbN..NbNaN  main -> main


In [25]:
## OLD: extraneous command as full path can be used as below
## # delete test local repos
## command cd "/tmp"

In [26]:
## OLD:
## rm -rf temp-git-test > /dev/null
## rm -rf temp-git-test-B > /dev/null
## Lorenzo-Note: useful for retaining directories for post-hoc analysis
rename-with-file-date /tmp/temp-git-test* > /dev/null
export GIT_NO_CONFIRM=$previous